In [1]:
%%configure
{"conf": {
    "spark.app.name":"oh-my-git_final",
     "driverMemory": "2000M",
    "executorMemory": "8G",
    "executorCores": 4,
    "numExecutors": 20
}}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
8242,application_1589299642358_2772,pyspark,idle,Link,Link,
8253,application_1589299642358_2785,pyspark,idle,Link,Link,
8259,application_1589299642358_2792,pyspark,idle,Link,Link,
8277,application_1589299642358_2812,pyspark,busy,Link,Link,
8280,application_1589299642358_2814,pyspark,idle,Link,Link,
8284,application_1589299642358_2816,pyspark,idle,Link,Link,
8292,application_1589299642358_2824,pyspark,idle,Link,Link,
8293,application_1589299642358_2825,pyspark,idle,Link,Link,
8294,application_1589299642358_2826,pyspark,idle,Link,Link,
8295,application_1589299642358_2827,pyspark,busy,Link,Link,


In [2]:
# Initialization

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
8299,application_1589299642358_2831,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# Import relevant libraries
from geopy import distance
import pyspark.sql.types as T
import pyspark.sql.functions  as F
from pyspark.sql.window import Window
import datetime
import networkx as nx
import pandas as pd
from itertools import islice, tee, izip, combinations

# Some constants
zurichHB = (47.378177, 8.540192)
cleveland_oh = (49.378177, 9.540192)
print(distance.distance(zurichHB, cleveland_oh).m)

# Useful user-defined functions
@F.udf(T.DoubleType())
def getDistToZurich(lat,lon):
    """
    Get distance to the Zurich HB in kilometers.
    """
    zurichHB = (47.378177, 8.540192)
    geo = (lat,lon)
    return distance.distance(zurichHB, geo).km

@F.udf(T.ArrayType(
       T.ArrayType(T.StructType([
    T.StructField("pair1", T.StringType(), False),
    T.StructField("pair2", T.StringType(), False)
]))))
def pairs(stop_ids,stop_sequences,arrival_times,departure_times):
    """
    Print the journey information in pairs.
    """
    stop_id_pairs = list(combinations(stop_ids,2))
    stop_sequence_pairs = list(combinations(stop_sequences,2))
    arrival_time_pairs = list(combinations(arrival_times,2))
    departure_time_pairs = list(combinations(departure_times,2))
    return [stop_id_pairs,stop_sequence_pairs,arrival_time_pairs,departure_time_pairs]

@F.udf(T.BooleanType())
def time_interval_udf(arrival_time_schedule,transfer_arrival_time):
    """
    Get a 2 minute time interval in order to do find relevant vehicles in the SBB data.
    """
    if arrival_time_schedule == "":
        return False
    ah = transfer_arrival_time.split(":")[0]
    am = transfer_arrival_time.split(":")[1]
    a = datetime.datetime(2019, 1, 1, int(ah), int(am))
    
    atsh = arrival_time_schedule.split(":")[0]
    atsm = arrival_time_schedule.split(":")[1]
    ats = datetime.datetime(2019, 1, 1, int(atsh), int(atsm))
    
    #Allow 120 seconds time difference
    if abs(ats-a).total_seconds() <= 120:
        return True
    else:
        return False
    
@F.udf(T.DoubleType())
def getDist(lat1,lon1,lat2,lon2):
    """
    Get distance between two coordinates in terms of meters.
    """
    geo1 = (lat1,lon1)
    geo2 = (lat2,lon2)
    dist = distance.distance(geo1, geo2).m
    return dist

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

234402.47499

In [4]:
# Read relevant tables
stop_times_df = spark.read.orc("hdfs:///data/sbb/timetables/orc/stop_times")
stops_df = spark.read.orc("hdfs:///data/sbb/timetables/orc/stops")
trips_df = spark.read.orc("hdfs:///data/sbb/timetables/orc/trips")
calendar_df = spark.read.orc("hdfs:///data/sbb/timetables/orc/calendar")
routes_df = spark.read.orc("hdfs:///data/sbb/timetables/orc/routes")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

First let's filter stations in 15km radius of Zurich HB

Then get the trips of these stops

In [5]:
stops_inradius_df= stops_df.withColumn("dist",getDistToZurich("stop_lat","stop_lon")).filter("dist <= 15")
stops_inradius_df = stops_inradius_df.drop("dist")
#Join the stops and trips
stops_joined_df = stop_times_df.join(stops_inradius_df,stop_times_df.stop_id == stops_inradius_df.stop_id ).drop(stops_inradius_df.stop_id)
stops_joined_df.drop("parent_station","location_type","pickup_type","drop_off_type").show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------+--------------+---------------+-------------+------------------+----------------+----------------+
|             trip_id|arrival_time|departure_time|        stop_id|stop_sequence|         stop_name|        stop_lat|        stop_lon|
+--------------------+------------+--------------+---------------+-------------+------------------+----------------+----------------+
|5.TA.1-1-A-j19-1.3.H|    02:42:00|      02:42:00|    8503305:0:3|            2|        Effretikon|  47.42593043029|8.68666388288076|
|5.TA.1-1-A-j19-1.3.H|    02:46:00|      02:46:00|    8503306:0:3|            3|         Dietlikon|47.4203145327083|8.61925430395105|
|5.TA.1-1-A-j19-1.3.H|    02:50:00|      02:50:00|    8503147:0:1|            4|         Stettbach| 47.397334167601|8.59613166853459|
|5.TA.1-1-A-j19-1.3.H|    02:55:00|      02:55:00|    8503003:0:1|            5|Zürich Stadelhofen|47.3667936853425|8.54846705955257|
|5.TA.1-1-A-j19-1.3.H|    02:58:00|      03:00:00|8503000:0:41

In [6]:
#Write this to use it later
#stops_joined_df.write.parquet("user/boecuego/stops_joined_df.parquet")

stops_joined_df = spark.read.parquet("user/boecuego/stops_joined_df.parquet")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Create Stop Connetions via Trips 

Here we are creating all possible stop connections reachable by public transports. For example, if we have stops_ids `1,2,3,4` for `trip X`, then connections are 1-2,1-3,1-4,2-3,2-4,3-4 (in other words combinations)

In [7]:
#First Collect all relevant information for each trip_id

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
df_grouped = stops_joined_df.groupBy("trip_id").agg(F.collect_list("stop_id").alias("stop_ids"),F.collect_list("stop_sequence").alias("stop_sequences"),\
                                      F.collect_list("arrival_time").alias("arrival_times"),\
                                      F.collect_list("departure_time").alias("departure_times"),\
                                      )                                    

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Then create combinations of these information 


In [9]:
df_grouped.cache()
df_grouped.take(1)
df_grouped.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

173184

In [10]:
df_pairs = df_grouped.withColumn("all_pairs",pairs("stop_ids","stop_sequences","arrival_times","departure_times"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
df_pairs.show(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|             trip_id|            stop_ids|      stop_sequences|       arrival_times|     departure_times|           all_pairs|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|1.TA.26-18-j19-1.1.H|[8591315, 8591142...|[1, 2, 3, 4, 5, 6...|[28:45:00, 28:46:...|[28:45:00, 28:46:...|[[[8591315, 85911...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 1 row

In [12]:
df_pairs_sep = df_pairs.withColumn("stop_id_pairs",F.col("all_pairs")[0])\
                        .withColumn("stop_sequence_pairs",F.col("all_pairs")[1])\
                        .withColumn("arrival_time_pairs",F.col("all_pairs")[2])\
                        .withColumn("departure_time_pairs",F.col("all_pairs")[3])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

For example this is how it looks for stop_id pairs, same applies for other columns.

In [13]:
df_pairs_sep.select("trip_id",F.explode("stop_id_pairs").alias("stop_id_pair")).withColumn("rown",F.monotonically_increasing_id()).show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------+----+
|             trip_id|      stop_id_pair|rown|
+--------------------+------------------+----+
|1.TA.26-18-j19-1.1.H|[8591315, 8591142]|   0|
|1.TA.26-18-j19-1.1.H|[8591315, 8530811]|   1|
|1.TA.26-18-j19-1.1.H|[8591315, 8591104]|   2|
|1.TA.26-18-j19-1.1.H|[8591315, 8591429]|   3|
|1.TA.26-18-j19-1.1.H|[8591315, 8591180]|   4|
|1.TA.26-18-j19-1.1.H|[8591315, 8530812]|   5|
|1.TA.26-18-j19-1.1.H|[8591315, 8591364]|   6|
|1.TA.26-18-j19-1.1.H|[8591315, 8530813]|   7|
|1.TA.26-18-j19-1.1.H|[8591142, 8530811]|   8|
|1.TA.26-18-j19-1.1.H|[8591142, 8591104]|   9|
+--------------------+------------------+----+
only showing top 10 rows

In [14]:
sti = df_pairs_sep.select("trip_id",F.explode("stop_id_pairs").alias("stop_id_pair")).withColumn("rown",F.monotonically_increasing_id()).alias("sti")
ssp = df_pairs_sep.select(F.explode("stop_sequence_pairs").alias("stop_sequence_pair"),F.monotonically_increasing_id().alias("rown")).alias("ssp")
atp = df_pairs_sep.select(F.explode("arrival_time_pairs").alias("arrival_time_pair"),F.monotonically_increasing_id().alias("rown")).alias("atp")
dtp = df_pairs_sep.select(F.explode("departure_time_pairs").alias("departure_time_pair"),F.monotonically_increasing_id().alias("rown")).alias("dtp")



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
#Uncomment this to run faster later

#ti.cache()
#ssp.cache()
#atp.cache()
#dtp.cache()
#sti.count()
#ssp.count()
#atp.count()
#dtp.count()

#Uncomment this to run faster later

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
joined = sti.join(ssp,sti.rown == ssp.rown).drop(ssp.rown).join(atp,sti.rown == atp.rown).drop(atp.rown).join(dtp,sti.rown == dtp.rown).drop(dtp.rown)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
#joined.write.mode("overwrite").parquet("user/boecuego/df_all_stop_connections.parquet")
joined = spark.read.parquet("user/boecuego/df_all_stop_connections.parquet")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Now let's see the final table

In [18]:
joined.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+----+------------------+--------------------+--------------------+
|             trip_id|        stop_id_pair|rown|stop_sequence_pair|   arrival_time_pair| departure_time_pair|
+--------------------+--------------------+----+------------------+--------------------+--------------------+
|1.TA.26-18-j19-1.1.H|  [8591142, 8591429]|  10|            [2, 5]|[28:46:00, 28:50:00]|[28:46:00, 28:50:00]|
|1005.TA.26-131-j1...|[8503855:0:F, 858...| 120|            [1, 7]|[16:14:00, 16:21:00]|[16:14:00, 16:21:00]|
|1022.TA.26-33E-j1...|  [8591230, 8591196]| 423|            [6, 7]|[18:44:00, 18:46:00]|[18:44:00, 18:46:00]|
|1022.TA.26-33E-j1...|  [8576198, 8576182]| 485|          [14, 17]|[18:55:00, 18:59:00]|[18:55:00, 18:59:00]|
|103.TA.26-1-A-j19...|[8503512:0:2, 850...| 500|          [14, 17]|[02:51:00, 03:00:00]|[02:51:00, 03:00:00]|
+--------------------+--------------------+----+------------------+--------------------+--------------------+
only showi

#### Filter for weekdays

Now we need to filter rows to obtain trips that are available Only on Weekdays (Monday, Tuesday, Wednesday, Thursday, Friday)

In [19]:
stop_times_df = spark.read.orc("hdfs:///data/sbb/timetables/orc/stop_times")
stops_df = spark.read.orc("hdfs:///data/sbb/timetables/orc/stops")
trips_df = spark.read.orc("hdfs:///data/sbb/timetables/orc/trips")
calendar_df = spark.read.orc("hdfs:///data/sbb/timetables/orc/calendar")
routes_df = spark.read.orc("hdfs:///data/sbb/timetables/orc/routes")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
df_service  =joined.join(trips_df,trips_df.trip_id ==joined.trip_id  )
df_days = df_service.join(calendar_df,calendar_df.service_id == df_service.service_id)
df_weekdays = df_days.filter("monday == 'true' AND tuesday == 'true' AND wednesday == 'true' AND thursday == 'true' AND friday == 'true' ")
cols = joined.columns
df_weekdays = df_weekdays.drop(trips_df.trip_id)
joined_filtered  = df_weekdays.select(*cols)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
#joined_filtered.write.mode("overwrite").parquet("user/boecuego/df_all_stop_connections_filtered.parquet")
joined_filtered = spark.read.parquet("user/boecuego/df_all_stop_connections_filtered.parquet")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
#Sort dataframe and selet only interested columns
joineds = joined_filtered.sort("rown").select("trip_id",F.col("stop_id_pair").pair1.alias("stop_id1")\
                        ,F.col("stop_id_pair").pair2.alias("stop_id2")\
                        ,F.col("stop_sequence_pair").pair1.alias("stop_sequence1")\
                        ,F.col("stop_sequence_pair").pair2.alias("stop_sequence2")\
                        ,F.col("arrival_time_pair").pair1.alias("arrival_time1")\
                        ,F.col("arrival_time_pair").pair2.alias("arrival_time2")\
                        ,F.col("departure_time_pair").pair1.alias("departure_time1")\
                        ,F.col("departure_time_pair").pair2.alias("departure_time2"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Finally we have all reachable stops

In [23]:
joineds.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----------+--------+--------------+--------------+-------------+-------------+---------------+---------------+
|             trip_id|   stop_id1|stop_id2|stop_sequence1|stop_sequence2|arrival_time1|arrival_time2|departure_time1|departure_time2|
+--------------------+-----------+--------+--------------+--------------+-------------+-------------+---------------+---------------+
|1005.TA.26-131-j1...|8503855:0:F| 8589111|             1|             2|     16:14:00|     16:15:00|       16:14:00|       16:15:00|
|1005.TA.26-131-j1...|8503855:0:F| 8573553|             1|             3|     16:14:00|     16:16:00|       16:14:00|       16:16:00|
|1005.TA.26-131-j1...|8503855:0:F| 8573554|             1|             4|     16:14:00|     16:18:00|       16:14:00|       16:18:00|
|1005.TA.26-131-j1...|8503855:0:F| 8573555|             1|             5|     16:14:00|     16:19:00|       16:14:00|       16:19:00|
|1005.TA.26-131-j1...|8503855:0:F| 8588985|             1|    

# Create Stop Connections via Walking

Get all walkable stop pair

**Conditions:**
- If distance(stop1,stop2) < 500 meters
- If stop in radius (15km from Zurich)

**Walking speed :** 50 meters in 1 minute

In [24]:
stops_inradius_df = stops_inradius_df.drop("dist")

stopDist = stops_inradius_df.alias("stops1").crossJoin(stops_inradius_df.alias("stops2"))\
.where(getDist("stops1.stop_lat","stops1.stop_lon","stops2.stop_lat","stops2.stop_lon")<= 500)\
.withColumn("dist",getDist("stops1.stop_lat","stops1.stop_lon","stops2.stop_lat","stops2.stop_lon"))


#Other option is this but don't know if it is way more efficient or more or less similiar
#stopDistFiltered = stops_inradius_df.alias("stops1").join(stops_inradius_df.alias("stops2")\
#,getDist("stops1.stop_lat","stops1.stop_lon","stops2.stop_lat","stops2.stop_lon")<=500)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
#Rename columns
walk_stops = stopDist.select(F.col("stops1.stop_id").alias("stop1_id")\
               ,F.col("stops1.stop_name").alias("stop1_name")\
               ,F.col("stops1.stop_lat").alias("stop1_lat")\
               ,F.col("stops1.stop_lon").alias("stop1_lon")\
               ,F.col("stops2.stop_id").alias("stop2_id")\
               ,F.col("stops2.stop_name").alias("stop2_name")\
               ,F.col("stops2.stop_lat").alias("stop2_lat")\
               ,F.col("stops2.stop_lon").alias("stop2_lon")\
               ,F.col("dist"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
#walk_stops.write.parquet("user/boecuego/walk_stops.parquet")

walk_stops = spark.read.parquet("user/boecuego/walk_stops.parquet")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
#Dist is distance between two stops
walk_stops.drop("stop1_lat","stop1_lon","stop2_lat","stop2_lon").show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------------------+-----------+--------------------+------------------+
|stop1_id|          stop1_name|   stop2_id|          stop2_name|              dist|
+--------+--------------------+-----------+--------------------+------------------+
| 8500926|Oetwil a.d.L., Sc...|    8500926|Oetwil a.d.L., Sc...|               0.0|
| 8500926|Oetwil a.d.L., Sc...|    8590616|Geroldswil, Schwe...|122.61607002692374|
| 8500926|Oetwil a.d.L., Sc...|    8590737|Oetwil an der Lim...|  300.671189772556|
| 8502186|Dietikon Stoffelbach|    8502186|Dietikon Stoffelbach|               0.0|
| 8502186|Dietikon Stoffelbach|8502186:0:1|Dietikon Stoffelbach| 6.761029676373361|
+--------+--------------------+-----------+--------------------+------------------+
only showing top 5 rows

# Create Graph 

In [28]:
df_trip_network_reasonable_main = spark.read.parquet("user/boecuego/df_all_stop_connections_filtered.parquet")
walk_stops_main = spark.read.parquet("user/boecuego/walk_stops.parquet")
df_trip_network_reasonable = df_trip_network_reasonable_main
walk_stops = walk_stops_main

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
df_trip_network_reasonable = df_trip_network_reasonable.sort("rown").select("trip_id",F.col("stop_id_pair").pair1.alias("stop_id1")\
                        ,F.col("stop_id_pair").pair2.alias("stop_id2")\
                        ,F.col("stop_sequence_pair").pair1.alias("stop_sequence1")\
                        ,F.col("stop_sequence_pair").pair2.alias("stop_sequence2")\
                        ,F.col("arrival_time_pair").pair1.alias("arrival_time1")\
                        ,F.col("arrival_time_pair").pair2.alias("arrival_time2")\
                        ,F.col("departure_time_pair").pair1.alias("departure_time1")\
                        ,F.col("departure_time_pair").pair2.alias("departure_time2"))
joineds = df_trip_network_reasonable

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

**Calculate the cost between two trip nodes:** (Arrival time of stop 2) - (Departure time of stop 1)

In [30]:
timeDiff = (F.unix_timestamp('arrival_time2', format="HH:mm:ss")
            - F.unix_timestamp('departure_time1', format="HH:mm:ss"))
df_trip_network_reasonable = df_trip_network_reasonable.withColumn("cost",F.ceil(timeDiff/60))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

**Calculate the cost between two walking nodes:** Distance/50 + 2minutes transfer delay

In [31]:
walk_stops = walk_stops.withColumn("cost", F.ceil(F.col("dist")/50)+2)
walk_stops = walk_stops.select(F.col('stop1_id').alias('stop_id1'), F.col('stop2_id').alias('stop_id2'), 'cost')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
df_trip_network_reasonables =  df_trip_network_reasonable.select("stop_id1","stop_id2","cost")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
df_trip_network_reasonables=df_trip_network_reasonables.withColumn("type",F.lit("trip"))
walk_stops = walk_stops.withColumn("type",F.lit("walk"))
unioned_df = df_trip_network_reasonables.union(walk_stops)
unioned_df = unioned_df.dropna()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

**Asumption for simplicity:** If there are multiple edges between two nodes we only get the edge with minimum

In [34]:
unioned_df_unique = unioned_df.groupBy("stop_id1","stop_id2").agg(F.min("cost").alias("cost"),F.collect_set("type").alias("types"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

**Asumption:** We want to penalize using too many transfers so we add extra 10minutes of cost to each edge

- A --> B --> C
- A --> C

we prioritize A -->C
We are also doing this because these two routes may belong to same trip and we don't want the intermediate stops

In [35]:
unioned_df_unique = unioned_df_unique.withColumn("cost",F.col("cost")+10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
pd_df = unioned_df_unique.toPandas()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Create the graph with networkx, we store cost (journey duration in minutes + penalty) and types (walking or public transport) in the edges

In [37]:
mygraph = nx.from_pandas_edgelist(pd_df, 'stop_id1', 'stop_id2',["cost","types"])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Find the probability

In [38]:
stops_df = spark.read.orc("hdfs:///data/sbb/timetables/orc/stops")
stops_inradius_df= stops_df.withColumn("dist",getDistToZurich("stop_lat","stop_lon")).filter("dist <= 15")

stops_inradius2 = set(stops_inradius_df.select("stop_id").distinct().rdd.map(lambda r: r[0]).collect())
stops_inradius = stops_inradius2.copy()
for s in stops_inradius2:
    if ":" in s:
        stops_inradius.add(s.split(":")[0])
sbb = spark.read.orc('/data/sbb/orc/istdaten')
sbb_inradius = sbb.where(F.col("bpuic").isin(stops_inradius))

sbb_inradius_weekdays = sbb_inradius.withColumn('date', F.from_unixtime(F.unix_timestamp('betriebstag', 'dd.MM.yyy')))\
    .withColumn("day",F.dayofweek("date"))\
        .filter("day BETWEEN 2 AND 6")


# rename the fields german -> english
fields = {
    'BETRIEBSTAG':'date',
    'FAHRT_BEZEICHNER':'trip_id',
    'PRODUKT_ID':'transport_type',
    'LINIEN_ID':'train_id',
    'LINIEN_TEXT':'line',
    'VERKEHRSMITTEL_TEXT':'train_type',
    'ZUSATZFAHRT_TF':'additional_trip',
    'FAELLT_AUS_TF':'trip_failed',
    'HALTESTELLEN_NAME':'stop_name',
    'BPUIC':'stop_id',
    'ANKUNFTSZEIT':'schedule_arrival',
    'AN_PROGNOSE':'real_arrival',
    'AN_PROGNOSE_STATUS':'arr_forecast_status',
    'ABFAHRTSZEIT':'schedule_dep',
    'AB_PROGNOSE':'real_dep',
    'AB_PROGNOSE_STATUS':'dep_forecast_status',
    'DURCHFAHRT_TF':'no_stop_here'
}
sbb_inradius_weekdays_english = sbb_inradius_weekdays.selectExpr([k + ' as ' + fields[k] for k in fields])

sbb_inradius_weekdays_english = sbb_inradius_weekdays_english.withColumn("arrival_time_schedule",\
                                                  sbb_inradius_weekdays_english.schedule_arrival.substr(12,10000))\
                                                    .withColumn("departure_time_schedule",\
                                                  sbb_inradius_weekdays_english.schedule_dep.substr(12,10000))\
                                                    .withColumn("arrival_time_real",\
                                                  sbb_inradius_weekdays_english.real_arrival.substr(12,10000))\
                                                    .withColumn("departure_time_real",\
                                                  sbb_inradius_weekdays_english.real_dep.substr(12,10000))
#sbb_inradius_weekdays_english.write.parquet("user/boecuego/sbb_inradius_weekdays_english.parquet")


sbb_inradius = spark.read.parquet("user/boecuego/sbb_inradius_weekdays_english.parquet")






FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Get only stop id's also exist between 13-17 May 2019

In [39]:
#Filter the trip_ids available only between 13-17 May


sbb_inradius_13_17_trips = sbb_inradius.filter("date <= '17.09.2019' AND date >= '13.09.2019' ").select("trip_id").distinct()
sbb_inradius_13_17_trips_select = sbb_inradius.join(F.broadcast(sbb_inradius_13_17_trips), sbb_inradius.trip_id == sbb_inradius_13_17_trips.trip_id)
sbb_inradius_13_17_trips_select = sbb_inradius_13_17_trips_select.drop(sbb_inradius.trip_id)
#sbb_inradius_13_17_trips_select.write.parquet("user/boecuego/sbb_inradius_13_17_trips_select2.parquet")

sbb_inradius_13_17_trips_select = spark.read.parquet("user/boecuego/sbb_inradius_13_17_trips_select2.parquet")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Collect also stop_ids of each trip 

For example -> trip_id1 : [stopid1,stopid2,...stopid7]

In [40]:
sbb_trip_stop_ids = sbb_inradius_13_17_trips_select.groupBy("trip_id","date")\
.agg(F.collect_list("stop_id").alias("stop_ids"))\
.groupBy("trip_id").agg(F.first("stop_ids").alias("stop_ids"))
#sbb_trip_stop_ids.write.parquet("user/boecuego/sbb_trip_stop_ids2.parquet")
sbb_trip_stop_ids = spark.read.parquet("user/boecuego/sbb_trip_stop_ids2.parquet")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
sbb_inradius_13_17_trips_wstops = sbb_inradius_13_17_trips_select\
.join(F.broadcast(sbb_trip_stop_ids), sbb_inradius_13_17_trips_select.trip_id == sbb_trip_stop_ids.trip_id)
sbb_inradius_13_17_trips_wstops = sbb_inradius_13_17_trips_wstops.drop(sbb_inradius_13_17_trips_select.trip_id)
#sbb_inradius_13_17_trips_wstops.write.parquet("user/boecuego/sbb_inradius_13_17_trips_wstops2.parquet")

sbb_inradius_13_17_trips_wstops = spark.read.parquet("user/boecuego/sbb_inradius_13_17_trips_wstops2.parquet")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
#sbb_inradius.write.parquet("user/boecuego/sbb_inradius_weekdays_engl.parquet")
#####
#sbb_inradius = spark.read.parquet("user/boecuego/sbb_inradius_weekdays_engl.parquet") #this is weekdays
###
#sbb_inradius = spark.read.parquet("user/boecuego/sbb_inradius_13_17_trips_select.parquet") #this is only trip_ids from 13_17
sbb_inradius_wostops = spark.read.parquet("user/boecuego/sbb_inradius_13_17_trips_wstops2.parquet")
sbb_inradius = spark.read.parquet("user/boecuego/sbb_inradius_13_17_trips_wstops2.parquet") #Also added stop sequences

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
stops_df.filter("stop_name LIKE '%Langmauerstrasse%'").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+----------------+----------------+-------------+--------------+
|stop_id|           stop_name|        stop_lat|        stop_lon|location_type|parent_station|
+-------+--------------------+----------------+----------------+-------------+--------------+
|8591244|Zürich, Langmauer...|47.3935396192995|8.54478396688717|             |              |
+-------+--------------------+----------------+----------------+-------------+--------------+

In [44]:
#!!! TO DO !!!!!
#Filter weekends, public holidays etc
#!!! TO DO !!!!!
def find_prob(stop_id,arrival_time_schedule):
    timeDiff = (F.unix_timestamp('arrival_time_real', format="HH:mm:ss")
            - F.unix_timestamp('arrival_time_schedule', format="HH:mm"))
    delays = sbb_inradius_wostops.filter((F.col('stop_id') == stop_id)& time_interval_udf(F.col('arrival_time_schedule'), F.lit(arrival_time_schedule))).withColumn("delay",F.floor(timeDiff/60)).groupBy("delay").count()
    #Find the total num
    #total_num = delays.groupBy().sum("count").collect()[0][0]
    #print(total_num)
    delays_pd = delays.toPandas()
    total_num = sum(delays_pd["count"].values)
    delays_pd["percentage"] = delays_pd["count"].map(lambda x:(x/float(total_num))*100)
    return delays_pd
    
def find_prob_wprevstop(stop_id,arrival_time_schedule,prev_stop_id):
    timeDiff = (F.unix_timestamp('arrival_time_real', format="HH:mm:ss")
            - F.unix_timestamp('arrival_time_schedule', format="HH:mm"))
    delays = sbb_inradius.filter((F.col("stop_id") == stop_id) \
    & (time_interval_udf("arrival_time_schedule",F.lit(arrival_time_schedule))) \
    & (F.array_contains(sbb_inradius.stop_ids,prev_stop_id ))).withColumn("delay",F.floor(timeDiff/60)).groupBy("delay").count()
    #Find the total num
    #total_num = delays.groupBy().sum("count").collect()[0][0]
    #print(total_num)
    delays_pd = delays.toPandas()
    total_num = sum(delays_pd["count"].values)
    delays_pd["percentage"] = delays_pd["count"].map(lambda x:(x/float(total_num))*100)
    return delays_pd    

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Find the shortest path

In [45]:
#Find the k shortest paths
def k_shortest_paths(G, source, target, k, weight):
    return list(islice(nx.shortest_simple_paths(G, source, target, weight=weight), k))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [46]:
for node in mygraph.nodes:
    print(node)
    break

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

8580301

Find **k** shortest path between two stops

In [47]:
src = "8591184"  #"8503000"
tgt  =  "8591244" #"8591049"

src_main = src.split(":")[0]
tgt_main = tgt.split(":")[0]
counter = 0
paths = []
for node in mygraph.nodes:
    if src_main in node:
        counter+=1
        
if counter > 1:
    k = 5
else:
    k=50
for node in mygraph.nodes:
    if src_main in node:
        p = k_shortest_paths(mygraph, node, tgt,k,weight = "cost")
        mygraph[src_main][node]["cost"] = 0
        for x in p:
            paths.append(x)


#paths = k_shortest_paths(mygraph, src_main, tgt_main,5,weight = "cost")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [48]:
len(paths)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

50

In [49]:

#This is a helper to get pairs in a list
def pairwise(iterable):
    "s -> (s0,s1), (s1,s2), (s2, s3), ..."
    a, b = tee(iterable)
    next(b, None)
    return izip(a, b)
#When converting a date to string this is a helper
def append_zero(x):
    if len(str(x)) == 1:
        return "0"+str(x)
    return str(x)
#Helper to remove adjecent walks
def remove_adjecent_walks(paths):
    no_adjecentwalk =[]
    for pr in paths:
        check = True
        for p1,p2 in pairwise(pairwise(pr)):

            if mygraph[p1[0]][p1[1]]["types"] == ["walk"] and mygraph[p2[0]][p2[1]]["types"] == ["walk"]:
                #print(counter)
                check = False

        if check:
            no_adjecentwalk.append(pr)
    return no_adjecentwalk

#Helper to reverse route
def get_reverse_route(paths):
    reversed_routes  = []
    for route in paths:
        lst = []
        for a in pairwise(route):
            lst.append(a)
        lst_reversed = lst[::-1]
        reversed_routes.append(lst_reversed)
    return reversed_routes

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

We don't want two consecutive walks, so filter them if any

In [50]:
no_adjecentwalk = remove_adjecent_walks(paths)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Since we know arrival time we need to **reverse** the path

In [51]:
reversed_routes  = get_reverse_route(no_adjecentwalk)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [52]:
joineds.cache()
joineds.take(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(trip_id=u'1005.TA.26-131-j19-1.9.H', stop_id1=u'8503855:0:F', stop_id2=u'8589111', stop_sequence1=u'1', stop_sequence2=u'2', arrival_time1=u'16:14:00', arrival_time2=u'16:15:00', departure_time1=u'16:14:00', departure_time2=u'16:15:00')]

In [53]:
#This is arrival_time
current_time = datetime.datetime(2019,1,1,12,30,0,0)
cut_off_time = datetime.datetime(2019,1,1,11,40,0,0)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

**Algorihm:** Starting from the last connection, we try to find most suitable  trips (if not possible walks) by comparing from timetables matching the arrival time and backtrace from there

In [54]:
notPossible = datetime.datetime(1999,1,1,12,12,0,0)
def find_possible_routes(reversed_routes,current_timez,cut_off_time):
    
    shorts = []
    longs = []
    notpossibles = []
    for route in reversed_routes:
        break_loop = False
        current_time = current_timez
        current_trip_id = ""
        
        route_with_timetable = []
        trip = []
        prints  = []
        for p1,p2 in route:
            #p2 arrival
            #p1 departure
            if mygraph[p1][p2]["types"] == ["walk"]:
                prints.append("Transfer with walking")
                prints.append(("after_walk",current_time))
                cost = mygraph[p1][p2]["cost"]
                if cost > 0:
                    current_time  = current_time - datetime.timedelta(minutes=cost-10)#-2
                else:
                    current_time = current_time
                prints.append(("before_walk",current_time))
                current_trip_id=""
                prints.append((p1,p2))
                continue
            else:
                #Get current times hour and minute
                hour = current_time.hour
                minute = current_time.minute

                #Convert it to string
                strtime = append_zero(hour)+":"+append_zero(minute)+":00"

                #Get closest trip to current time with matching stop_ids
                if current_trip_id != "":
                    trip = joineds.filter(F.col("trip_id") == current_trip_id)\
                    .filter("stop_id1 == '"+p1+"'"+" AND "+"stop_id2 == '"+p2+"'").take(1)
               #trip = df_trip_network_reasonable2.filter("prev_stop_id == '"+p1+"'"+" AND "+"stop_id == '"+p2+"'").filter("arrival_time <='"+strtime+"'").sort(F.desc("arrival_time")).take(1)[0]

                #If previous trip_id was different from this trip's id then put 2 minute waiting time
                if trip == [] or current_trip_id == "":

                    #2 min waiting
                    if current_trip_id != "":
                        current_time = current_time - datetime.timedelta(minutes = 2)
                        prints.append("Transfer")


                    #Get hours, minutes
                    hour = current_time.hour
                    minute = current_time.minute
                    strtime = append_zero(hour)+":"+append_zero(minute)+":00"

                    #With new current time get new trip
                    trip = joineds.filter("stop_id1 == '"+p1+"'"+" AND "+"stop_id2 == '"+p2+"'").filter("arrival_time2 <='"+strtime+"'").sort(F.desc("arrival_time2")).take(1)
                    #update current trip_id
                try:
                    trip = trip[0]
                except:
                    print("Encountered a route that is not possible currently",(p1,p2))
                    current_time = notPossible
                    break
                current_trip_id  = trip.trip_id

                #Update current time
                prints.append((trip.trip_id,trip.arrival_time1,trip.departure_time1,trip.arrival_time2,trip.departure_time2))
                hms = trip.departure_time1.split(":")
                h = int(hms[0])
                m = int(hms[1])
                current_time = datetime.datetime(2019,1,1,h,m,0,0)

            prints.append((p1,p2))
            if break_loop:
                prints = []
                break
        if current_time >= cut_off_time:
            shorts.append(prints)
            shorts.append(current_time)
            shorts.append("---------------------------------------------------------------")
        elif current_time == notPossible:
            notpossibles.append(prints)
            notpossibles.append(current_time)
            notpossibles.append("---------------------------------------------------------------")
        else:
            longs.append(prints)
            longs.append("---------------------------------------------------------------")
    return shorts,longs

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [55]:
good_routes,bad_routes = find_possible_routes(reversed_routes,current_time,cut_off_time)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

('Encountered a route that is not possible currently', (u'8591184', u'8591058'))
('Encountered a route that is not possible currently', (u'8503059', '8591244'))
('Encountered a route that is not possible currently', (u'8591184', u'8591404'))
('Encountered a route that is not possible currently', (u'8530813', '8591244'))
('Encountered a route that is not possible currently', (u'8503083', '8591244'))
('Encountered a route that is not possible currently', (u'8591199', '8591244'))

In [56]:
for element in good_routes:
        if type(element) == type([]):
            for r in element:
                print(r)
        else:
            print element

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(u'253.TA.26-9-B-j19-1.1.R', u'12:02:00', u'12:03:00', u'12:23:00', u'12:23:00')
(u'8591381', '8591244')
Transfer
(u'3718.TA.26-8-C-j19-1.27.H', u'11:54:00', u'11:54:00', u'11:56:00', u'11:56:00')
(u'8591184', u'8591381')
2019-01-01 11:54:00
---------------------------------------------------------------
(u'253.TA.26-9-B-j19-1.1.R', u'12:12:00', u'12:12:00', u'12:23:00', u'12:23:00')
(u'8576193', '8591244')
Transfer
(u'3718.TA.26-8-C-j19-1.27.H', u'11:54:00', u'11:54:00', u'12:05:00', u'12:05:00')
(u'8591184', u'8576193')
2019-01-01 11:54:00
---------------------------------------------------------------
(u'253.TA.26-9-B-j19-1.1.R', u'12:07:00', u'12:07:00', u'12:23:00', u'12:23:00')
(u'8591299', '8591244')
Transfer
(u'3718.TA.26-8-C-j19-1.27.H', u'11:54:00', u'11:54:00', u'12:01:00', u'12:01:00')
(u'8591184', u'8591299')
2019-01-01 11:54:00
---------------------------------------------------------------
(u'253.TA.26-9-B-j19-1.1.R', u'12:10:00', u'12:10:00', u'12:23:00', u'12:23:00')
(

In [57]:
for element in bad_routes:
        if type(element) == type([]):
            for r in element:
                print(r)
        else:
            print element

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(u'2078.TA.26-6-B-j19-1.16.R', u'08:51:00', u'08:51:00', u'09:08:00', u'09:08:00')
(u'8591384', '8591244')
Transfer
(u'3755.TA.26-8-C-j19-1.27.H', u'08:39:00', u'08:39:00', u'08:44:00', u'08:44:00')
(u'8591184', u'8591384')
---------------------------------------------------------------
(u'2078.TA.26-6-B-j19-1.16.R', u'08:56:00', u'08:56:00', u'09:08:00', u'09:08:00')
(u'8591067', '8591244')
Transfer
(u'1851.TA.26-17-j19-1.31.R', u'06:15:00', u'06:15:00', u'06:20:00', u'06:20:00')
(u'8591381', u'8591067')
Transfer
(u'3816.TA.26-8-C-j19-1.27.H', u'06:02:00', u'06:02:00', u'06:03:00', u'06:03:00')
(u'8591184', u'8591381')
---------------------------------------------------------------

Find best route(s)

**Assumption:** Best route(s) is(are) the route(s) with latest departure(s)

In [58]:
maxt = datetime.datetime(2019, 1, 1, 0,0)

index = -1
start_times  = []
for i,t in enumerate(good_routes):
    if type(t) == type(datetime.datetime(2019, 1, 1, 11, 56)):
        start_times.append((t,i))
        if t > maxt:
            maxt = t
            index = i
start_times_reversed = sorted(start_times, key=lambda tup: tup[0],reverse=True)
best_routes = []
for time,index in start_times_reversed[:10]:
    route = good_routes[index-1:index]
    best_routes.append(route[0])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Print best route

In [59]:
for r in best_routes:
    print("******"*10)
    for x in r:
        print x

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

************************************************************
(u'3192.TA.26-9-B-j19-1.25.H', u'12:26:00', u'12:26:00', u'12:29:00', u'12:29:00')
(u'8591276', '8591244')
Transfer
(u'1177.TA.26-69-j19-1.4.H', u'12:16:00', u'12:16:00', u'12:19:00', u'12:19:00')
(u'8591101', u'8591276')
Transfer
(u'629.TA.26-32-j19-1.10.R', u'12:03:00', u'12:03:00', u'12:14:00', u'12:14:00')
(u'8591184', u'8591101')
************************************************************
(u'1460.TA.26-10-j19-1.11.R', u'12:12:00', u'12:12:00', u'12:25:00', u'12:25:00')
(u'8591262', '8591244')
Transfer
(u'868.TA.26-14-A-j19-1.10.R', u'12:05:00', u'12:07:00', u'12:10:00', u'12:10:00')
(u'8591381', u'8591262')
Transfer
(u'3717.TA.26-8-C-j19-1.27.H', u'12:02:00', u'12:02:00', u'12:03:00', u'12:03:00')
(u'8591184', u'8591381')
************************************************************
(u'3192.TA.26-9-B-j19-1.25.H', u'12:26:00', u'12:26:00', u'12:29:00', u'12:29:00')
(u'8591276', '8591244')
Transfer
(u'868.TA.26-14-A-j19-1.

In [60]:
only_routes = good_routes[::3]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Find probabiliy for the good routes

For each transfer, we calculate the probability of not missing the transfer vehicle. Assume that a passanger does $n$ transfers for the planned journey. Then, the probability of arriving in time is:

\begin{align}
Pr[\textrm{Arriving in time with the found route}] & = Pr[\textrm{Not missing any transfers}] \cdot Pr[\textrm{Arriving the final station in time}] \\
                                                   & = \prod_{i=0}^{n-1} Pr[\textrm{Not missing transfer i}] \cdot Pr[\textrm{Arriving the final station in time}]
\end{align}

where the second equality follows from the assumption that **the delays of transports are independent of each other**.

In [61]:
## !!!!!!! TO CHECK !!!!!!!!
# This is currently not checking the probabiliy of desired arival time Also need to check that
#!!!!!!! TO CHECK !!!!!!!!

def calculate_probabilities(desired_arrival_time,best_routes):
    
    for r in best_routes:
        overall_prob = 1
        print("*"*100)

        for i in range(len(r)):
            # This case corresponds to the Pr[Arriving the final station in time] case
            # That is why the index is zero and the final means of travelling should not be walking but a public transport
            if i == 0 and r[i]!="Transfer with walking":
                las_arrival_time = r[i][3]
                ah = las_arrival_time.split(":")[0]
                am = las_arrival_time.split(":")[1]
                a = datetime.datetime(2019, 1, 1, int(ah), int(am))
                last_station = r[i+1][1]
                
                allowed_delay = (desired_arrival_time-a).total_seconds()/60
                prob_dist =find_prob(last_station,las_arrival_time[:-3])
                probability_to_catch = sum(prob_dist[prob_dist["delay"]<= allowed_delay].percentage.values)
                overall_prob = (overall_prob*probability_to_catch)/100
                print("Prob to arrive on time: ",probability_to_catch)

            try:
                if r[i] == "Transfer with walking":

                    transfer_departure_time = r[i-2][2]
                    dh = transfer_departure_time.split(":")[0]
                    dm = transfer_departure_time.split(":")[1]
                    d = datetime.datetime(2019, 1, 1, int(dh), int(dm))
                    #think this
                    transfer_station =  r[i+5][1].split(":")[0]

                    walking_min = (r[i+1][1] - r[i+2][1]).total_seconds()/60

                    #think this
                    transfer_arrival_time = r[i+4][3]
            

                    ah = transfer_arrival_time.split(":")[0]
                    am = transfer_arrival_time.split(":")[1]
                    a = datetime.datetime(2019, 1, 1, int(ah), int(am))
                    allowed_delay = (d-a).total_seconds()/60 - walking_min
                    print(transfer_station,transfer_arrival_time)
                    prob_dist =find_prob(transfer_station,transfer_arrival_time[:-3])

                    probability_to_catch = sum(prob_dist[prob_dist["delay"]<= allowed_delay].percentage.values)
                    print("For station ",transfer_station,"probability to catch trip ",r[i-2][0]," : with walking transfer probability",
                           probability_to_catch,"departing at ",transfer_departure_time)
                    print("Last vehicle arrived at: ",transfer_arrival_time)
                    overall_prob = (overall_prob*probability_to_catch)/100
                
                elif r[i] == "Transfer":
                    transfer_departure_time = r[i-2][2]
                    dh = transfer_departure_time.split(":")[0]
                    dm = transfer_departure_time.split(":")[1]
                    d = datetime.datetime(2019, 1, 1, int(dh), int(dm))
                    transfer_station =  r[i-1][1]
                    
                    transfer_arrival_time = r[i+1][3]
                    
                    ah = transfer_arrival_time.split(":")[0]
                    am = transfer_arrival_time.split(":")[1]
                    a = datetime.datetime(2019, 1, 1, int(ah), int(am))
                    allowed_delay = (d-a).total_seconds()/60
                    prob_dist =find_prob(transfer_station,transfer_arrival_time[:-3])
                    
                    probability_to_catch = sum(prob_dist[prob_dist["delay"]<= allowed_delay].percentage.values)
                    print("For station ",transfer_station,"probability to catch trip ",r[i-2][0]," :direct transfer probability",
                           probability_to_catch,"departing at ",transfer_departure_time)
                    print("Last vehicle arrived at: ",transfer_arrival_time)
                    overall_prob = (overall_prob*probability_to_catch)/100

            except:
                print("Except",r[i],"First or last trip is walking so no transfer")
        print("Overal probability: ",overall_prob)         

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [62]:
calculate_probabilities(current_time,best_routes)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

****************************************************************************************************
('Prob to arrive on time: ', 89.60893854748605)
('For station ', '8591244', 'probability to catch trip ', u'3192.TA.26-9-B-j19-1.25.H', ' :direct transfer probability', 100.0, 'departing at ', u'12:26:00')
('Last vehicle arrived at: ', u'12:19:00')
('For station ', u'8591276', 'probability to catch trip ', u'1177.TA.26-69-j19-1.4.H', ' :direct transfer probability', 95.956012770486, 'departing at ', u'12:16:00')
('Last vehicle arrived at: ', u'12:14:00')
('Overal probability: ', 0.8598516451612267)
****************************************************************************************************
('Prob to arrive on time: ', 98.80597014925374)
('For station ', '8591244', 'probability to catch trip ', u'1460.TA.26-10-j19-1.11.R', ' :direct transfer probability', 90.06721433905899, 'departing at ', u'12:12:00')
('Last vehicle arrived at: ', u'12:10:00')
('For station ', u'8591262', 'prob